<a href="https://www.kaggle.com/code/jmmrcp/briefing-ai?scriptVersionId=282804909" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🤖 AI Executive Assistant: Sistema Multi-Agente con CrewAI

## 🎯 Objetivo del Proyecto
Este proyecto implementa un **Asistente Ejecutivo Personal** totalmente autónomo utilizando **CrewAI**. El objetivo es automatizar el "Briefing Matutino" (Morning Briefing) que recibiría un alto ejecutivo, consolidando información de múltiples fuentes heterogéneas en un único reporte accionable.

A diferencia de los asistentes simples que solo responden preguntas, este **Equipo de Agentes (Crew)** actúa proactivamente:
1.  **Se conecta a servicios reales:** Gmail, Google Calendar, Google Tasks.
2.  **Analiza el mercado:** Obtiene datos bursátiles y noticias en tiempo real.
3.  **Monitoriza el mundo físico:** Revisa el estado del transporte público local.
4.  **Notifica multicanal:** Envía el reporte final vía Telegram, WhatsApp y Pushover.

## 🏗️ Arquitectura
El sistema se basa en una orquestación secuencial de agentes especializados, donde cada uno tiene un rol, una "historia de fondo" (backstory) y un set de herramientas (Tools) específicas.

In [1]:
# ==========================
# Librerías estándar de Python
# ==========================
import os                           # Operaciones con el sistema de archivos
import sys                          # Acceso a funciones y variables del intérprete
import subprocess                   # Ejecución de procesos del sistema
import importlib                    # Carga dinámica de módulos
import logging                      # Configuración y manejo de logs


In [2]:
# Configuración de Logging
log_level = os.environ.get('LOG_LEVEL', 'INFO').upper()
logging.basicConfig(
    format='%(asctime)s - [%(levelname)s] - %(message)s',
    level=getattr(logging, log_level),
    force=True
)
logger = logging.getLogger(__name__)

In [3]:
def install_and_check_dependencies():
    """
    Verifica e instala las dependencias necesarias para el Asistente Ejecutivo.
    Mapea el nombre del paquete en PIP con el nombre del módulo en Python.
    """
    print("⚙️ Instalando Tesseract OCR (Sistema Linux)...")
    # Esto es necesario para que funcione pytesseract en Kaggle
    os.system("apt-get update && apt-get install -y tesseract-ocr libtesseract-dev")

    # Diccionario: "nombre-en-pip": "nombre-del-modulo-python"
    dependencies = {
        "feedparser": "feedparser",
        "crewai": "crewai",
        "yfinance": "yfinance",
        "nest_asyncio": "nest_asyncio",
        "google-auth-oauthlib": "google_auth_oauthlib",
        "google-api-python-client": "googleapiclient",
        "twilio": "twilio",
        "pytesseract": "pytesseract",
        "requests": "requests",
        "Pillow": "Pillow",
        "beautifulsoup4": "BeautifulSoup",
    }

    logger.info("🔍 Verificando dependencias del sistema...")

    cambios_realizados = False

    for package, module in dependencies.items():
        try:
            importlib.import_module(module)
            # Si llegamos aquí, el módulo existe, no hacemos nada (o logueamos debug)
        except ImportError:
            logger.info(f"⬇️  Instalando {package}...")
            try:
                # Ejecuta pip install en modo silencioso (-q)
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
                cambios_realizados = True
                logger.info(f"✅ {package} instalado correctamente.")
            except subprocess.CalledProcessError:
                logger.error(f"❌ Error crítico al instalar {package}.")

    if cambios_realizados:
        logger.info("🎉 Todas las dependencias han sido instaladas/verificadas.")
    else:
        logger.info("✅ Todo está al día. No se requirieron instalaciones.")

In [4]:
install_and_check_dependencies()

⚙️ Instalando Tesseract OCR (Sistema Linux)...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,836 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports In

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)



Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 180 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.5 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Fetched 3,743 kB in 4s (970 kB/s)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../liba

2025-11-30 07:42:58,620 - [INFO] - 🔍 Verificando dependencias del sistema...
2025-11-30 07:42:58,622 - [INFO] - ⬇️  Instalando feedparser...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.7 MB/s eta 0:00:00


2025-11-30 07:43:05,322 - [INFO] - ✅ feedparser instalado correctamente.
2025-11-30 07:43:05,326 - [INFO] - ⬇️  Instalando crewai...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.9/642.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
google-colab 1.0.0 requires tornado==6.4.2, but you have tornado 6.5.2 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.0 which is inco

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.9 MB/s eta 0:00:00


2025-11-30 07:43:33,219 - [INFO] - ✅ twilio instalado correctamente.
2025-11-30 07:43:33,232 - [INFO] - ⬇️  Instalando Pillow...
2025-11-30 07:43:35,939 - [INFO] - ✅ Pillow instalado correctamente.
2025-11-30 07:43:35,941 - [INFO] - ⬇️  Instalando beautifulsoup4...
2025-11-30 07:43:38,565 - [INFO] - ✅ beautifulsoup4 instalado correctamente.
2025-11-30 07:43:38,566 - [INFO] - 🎉 Todas las dependencias han sido instaladas/verificadas.


In [5]:
# ==========================
# Librerías estándar de Python
# ==========================
import json                     # Manejo de datos en formato JSON
import urllib.parse             # Funciones para parsear y manipular URLs
from datetime import datetime, timedelta  # Manejo de fechas y tiempos
from typing import List, Dict, Optional, Any  # Tipado estático para anotaciones

# ==========================
# Librerías externas para HTTP y datos
# ==========================
import requests                 # Realizar peticiones HTTP
import feedparser               # Parseo de feeds RSS/Atom
import yfinance as yf           # Acceso a datos financieros (Yahoo Finance)
import pandas as pd             # Manipulación y análisis de datos

# ==========================
# Google APIs
# ==========================
from google.oauth2.credentials import Credentials       # Autenticación OAuth2
from google.auth.transport.requests import Request      # Transporte para OAuth
from google_auth_oauthlib.flow import InstalledAppFlow  # Flujo de autenticación
from googleapiclient.discovery import build             # Construcción de clientes API
from google.colab import userdata                       # Acceso a datos en Google Colab
from kaggle_secrets import UserSecretsClient
# ==========================
# CrewAI (Agentes y Tareas)
# ==========================
from crewai import Crew, Agent, Task, LLM          # Componentes principales de CrewAI
from crewai.tools import tool                      # Decorador para definir herramientas

# ==========================
# Twilio (Mensajería)
# ==========================
from twilio.rest import Client                     # Cliente para enviar SMS y llamadas

# ==========================
# Visualización
# ==========================
from IPython.display import display, Markdown      # Mostrar contenido en notebooks

# ==========================
# Procesamiento de HTML y OCR
# ==========================
from bs4 import BeautifulSoup                      # Parseo de HTML
from PIL import Image                              # Manipulación de imágenes
import pytesseract                                 # OCR (Reconocimiento óptico de caracteres)
from io import BytesIO                             # Manejo de datos binarios en memoria

# ==========================
# Utilidades adicionales
# ==========================
from urllib.parse import urlparse                  # Parseo de URLs


2025-11-30 07:43:49,314 - [INFO] - Using config path: /root/.config/crewai/settings.json


# CONFIGURACIÓN E INICIALIZACIÓN


In [6]:
# Scopes de Google necesarios
SCOPES = [
    "https://www.googleapis.com/auth/calendar.readonly",
    "https://www.googleapis.com/auth/tasks.readonly",
    "https://www.googleapis.com/auth/gmail.readonly"
]

In [7]:
# Variables de entorno (Cargadas de manera segura)
# NOTA: En producción, usa userdata.get() o variables de sistema. NO hardcodees aquí.
DRY_RUN = os.environ.get("DRY_RUN", "0") == "1"

In [8]:
# Intentamos cargar credenciales de userdata (Colab) o entorno
try:
    os.environ["PUSHOVER_USER"] = UserSecretsClient().get_secret('PUSHOVER_USER') or os.environ.get('PUSHOVER_USER', '')
    os.environ["PUSHOVER_TOKEN"] = UserSecretsClient().get_secret('PUSHOVER_TOKEN') or os.environ.get('PUSHOVER_TOKEN', '')
    os.environ["GOOGLE_API_KEY"] = UserSecretsClient().get_secret('GOOGLE_API_KEY') or os.environ.get('GOOGLE_API_KEY')
    os.environ["TELEGRAM_TOKEN"] = UserSecretsClient().get_secret('TELEGRAM_TOKEN') or os.environ.get('TELEGRAM_TOKEN', '')
    os.environ["TELEGRAM_CHAT_ID"] = UserSecretsClient().get_secret('TELEGRAM_CHAT_ID') or os.environ.get('TELEGRAM_CHAT_ID', '')
    #os.environ["WHATSAPP_PHONE"] = userdata.get('WHATSAPP_PHONE') or os.environ.get('WHATSAPP_PHONE', '')
    #os.environ["WHATSAPP_API_KEY"] = userdata.get('WHATSAPP_API_KEY') or os.environ.get('WHATSAPP_API_KEY', '')
except Exception as e:
    logger.warning(f"Advertencia cargando secretos: {e}. Asegúrate de tener las variables de entorno configuradas.")

In [9]:
# Límites para evitar errores de API
RPM_LIMIT = 10

In [10]:
# Campos financieros a extraer
CLAVES_PRINCIPALES = [
    "symbol", "shortName", "currency", "exchange",
    "currentPrice", "regularMarketPrice", "previousClose",
    "regularMarketChangePercent", "dayLow", "dayHigh",
    "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "marketCap", "dividendYield"
]

# 2. FUNCIONES DE UTILIDAD Y AUTENTICACIÓN

In [11]:
%cp /kaggle/input/token-google/token.json /kaggle/working/token.json

In [12]:
def get_llm():
    """
    Retorna una instancia configurada del LLM (Gemini Flash).

    Returns:
        LLM: Instancia lista para usar en CrewAI.
    """
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY no está configurada. Verifica tus secretos.")

    return LLM(
        model="gemini/gemini-flash-latest", # Nombre del modelo actualizado
        verbose=True,
        temperature=0, # Temperatura baja para reducir alucinaciones
        google_api_key=api_key
    )

In [13]:
def authenticate_google_services() -> Optional[Credentials]:
    """
    Maneja el flujo de autenticación OAuth 2.0 para servicios de Google.

    Returns:
        Credentials: Objeto de credenciales válido o None si falla.
    """
    creds = None
    token_file = '/kaggle/working/token.json'

    if os.path.exists(token_file):
         creds = Credentials.from_authorized_user_file(token_file, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
                logger.info("🔄 Token refrescado automáticamente")
            except Exception:
                logger.warning("⚠️ Token expirado y no renovable. Re-autenticando...")
                creds = None

        if not creds:
            if not os.path.exists('credentials.json'):
                logger.error("❌ Faltan 'credentials.json'. Descárgalo de Google Cloud Console.")
                return None

            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            # run_local_server puede fallar en entornos headless (nube), usar con precaución
            creds = flow.run_local_server(port=0)

        # Guardar token para la próxima vez
        with open(token_file, 'w') as token:
            token.write(creds.to_json())

    return creds

# 3. DEFINICIÓN DE HERRAMIENTAS (TOOLS)

In [14]:
@tool("Enviar Telegram")
def send_telegram(message: str) -> str:
    """
    Envía el reporte a Telegram. Es más rápido y fiable que WhatsApp.
    Requiere: TELEGRAM_TOKEN, TELEGRAM_CHAT_ID.
    """
    token = os.environ.get("TELEGRAM_TOKEN")
    chat_id = os.environ.get("TELEGRAM_CHAT_ID")

    if not token or not chat_id:
        return "Error: Faltan credenciales de Telegram."

    url = f"https://api.telegram.org/bot{token}/sendMessage"

    # Telegram soporta Markdown si quieres negritas
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"
    }

    try:
        response = requests.post(url, json=payload, timeout=10)
        if response.status_code == 200:
            return "Telegram enviado."
        return f"Error Telegram: {response.text}"
    except Exception as e:
        return f"Excepción Telegram: {e}"

In [15]:
@tool("Noticias RSS")
def get_financial_news(busqueda: str) -> str:
    """
    Busca noticias recientes en Google News (RSS).
    Args:
        busqueda (str): Término a buscar (ej: "El Corte Inglés").
    Returns:
        str: JSON con lista de noticias (título, link, fecha, fuente).
    """
    base_url = "https://news.google.com/rss/search"
    params = {"q": busqueda, "hl": "es-ES", "gl": "ES", "ceid": "ES:es"}

    # Headers vitales para evitar bloqueo (403/429)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/115.0.0.0 Safari/537.36"}

    try:
        response = requests.get(base_url, params=params, headers=headers, timeout=10)
        response.raise_for_status()

        f = feedparser.parse(response.content)
        noticias_procesadas = []

        if not f.entries:
            return json.dumps({"news": [], "mensaje": f"Sin noticias para '{busqueda}'."}, ensure_ascii=False)

        for entry in f.entries[:5]: # Top 5 noticias
            noticias_procesadas.append({
                "titulo": entry.title,
                "link": entry.link,
                "fecha": entry.get("published", "Fecha desconocida"),
                "fuente": entry.get("source", {}).get("title", "Google News")
            })

        return json.dumps({"news": noticias_procesadas}, indent=2, ensure_ascii=False)

    except Exception as e:
        logger.error(f"Error noticias: {e}")
        return json.dumps({"error": str(e), "news": []}, ensure_ascii=False)

In [16]:
@tool("Bolsa")
def get_stock_price(symbol: str) -> str:
    """
    Obtiene datos financieros filtrados de Yahoo Finance.
    Args:
        symbol (str): Ticker (ej: "REP.MC").
    Returns:
        str: JSON con datos clave de mercado.
    """
    try:
        symbol = symbol.strip().upper()
        t = yf.Ticker(symbol)
        info = t.info

        if not info or 'currentPrice' not in info and 'regularMarketPrice' not in info:
             return json.dumps({"error": "Ticker inválido", "stock": {}}, ensure_ascii=False)

        datos_filtrados = {k: info.get(k) for k in CLAVES_PRINCIPALES if info.get(k) is not None}
        return json.dumps({"stock": datos_filtrados}, indent=2, ensure_ascii=False)

    except Exception as e:
        logger.error(f"Error bolsa: {e}")
        return json.dumps({"error": str(e), "stock": {}}, ensure_ascii=False)

In [17]:
@tool("Enviar Pushover")
def send_pushover(msg: str) -> str:
    """Envía notificaciones vía Pushover."""
    if DRY_RUN: return "Simulación: Enviado."

    user = os.environ.get("PUSHOVER_USER")
    token = os.environ.get("PUSHOVER_TOKEN")

    if not user or not token: return "Error: Faltan credenciales Pushover."

    try:
        resp = requests.post(
            "https://api.pushover.net/1/messages.json",
            data={"token": token, "user": user, "message": msg, "title": "Briefing IA"},
            timeout=10
        )
        return "Enviado." if resp.status_code == 200 else f"Error API: {resp.text}"
    except Exception as e:
        return f"Excepción Pushover: {e}"

In [18]:
@tool("Enviar WhatsApp")
def send_whatsapp(message: str) -> str:
    """
    Envía un mensaje de WhatsApp utilizando la API de Twilio.
    Requiere las variables de entorno: TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN,
    TWILIO_FROM_NUMBER y WHATSAPP_PHONE.
    """
    # 1. Verificar si es una simulación (DRY_RUN)
    if os.environ.get("DRY_RUN", "0") == "1":
        logger.info(f"Simulación Twilio: {message[:50]}...")
        return json.dumps({"resultado": "Simulación: Enviado (DRY_RUN)"}, ensure_ascii=False)

    # 2. Obtener credenciales
    account_sid = os.environ.get("TWILIO_ACCOUNT_SID")
    auth_token = os.environ.get("TWILIO_AUTH_TOKEN")
    from_number = os.environ.get("TWILIO_FROM_NUMBER") # Ej: +14155238886
    to_number = os.environ.get("WHATSAPP_PHONE")       # Tu número personal

    # Validación de seguridad
    if not all([account_sid, auth_token, from_number, to_number]):
        logger.error("Faltan credenciales de Twilio.")
        return json.dumps({"error": "Configuración incompleta de Twilio"}, ensure_ascii=False)

    try:
        # 3. Inicializar Cliente Twilio
        client = Client(account_sid, auth_token)

        # Twilio exige el prefijo 'whatsapp:' antes del número E.164
        # Nos aseguramos de que el prefijo esté presente
        if not from_number.startswith("whatsapp:"):
            from_number = f"whatsapp:{from_number}"

        if not to_number.startswith("whatsapp:"):
            to_number = f"whatsapp:{to_number}"

        # 4. Enviar Mensaje
        # Nota: Twilio trunca mensajes muy largos (>1600 caracteres),
        # pero es seguro cortar a 1000 para asegurar entrega rápida.
        msg = client.messages.create(
            body=message[:1500],
            from_=from_number,
            to=to_number
        )

        logger.info(f"WhatsApp enviado. SID: {msg.sid}")
        return json.dumps({
            "resultado": "Enviado correctamente",
            "id_mensaje": msg.sid
        }, ensure_ascii=False)

    except Exception as e:
        logger.error(f"Error enviando WhatsApp con Twilio: {e}")
        return json.dumps({"error": str(e)}, ensure_ascii=False)

In [19]:
@tool("Enviar WhatsApp")
def send_whatsapp_old(message: str) -> str:
    """Envía WhatsApp vía CallMeBot."""
    if DRY_RUN: return "Simulación: Enviado."

    phone = os.environ.get("WHATSAPP_PHONE")
    apikey = os.environ.get("WHATSAPP_API_KEY")

    if not phone or not apikey: return json.dumps({"error": "Faltan credenciales WhatsApp"})

    try:
        # Usamos params para codificación automática y segura
        url = "https://api.callmebot.com/whatsapp.php"
        params = {"phone": phone, "text": message[:1000], "apikey": apikey} # Truncar a 1000 chars

        response = requests.get(url, params=params, timeout=15)

        if response.status_code == 200:
            return json.dumps({"resultado": "Enviado correctamente"}, ensure_ascii=False)
        return json.dumps({"error": f"API Error: {response.text}"}, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)}, ensure_ascii=False)

In [20]:
@tool("Enviar WhatsApp Meta")
def send_whatsapp_meta(message: str) -> str:
    """
    Envía mensaje usando la API oficial de WhatsApp Cloud (Meta).
    Requiere: META_TOKEN, META_PHONE_ID, WHATSAPP_PHONE.
    """
    token = os.environ.get("META_TOKEN")
    phone_id = os.environ.get("META_PHONE_ID")
    to_number = os.environ.get("WHATSAPP_PHONE") # Debe incluir código país sin '+' (ej: 34666...)

    if not all([token, phone_id, to_number]):
        return "Error: Faltan credenciales de Meta."

    url = f"https://graph.facebook.com/v22.0/{phone_id}/messages"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    # Payload para mensaje de texto simple
    data = {
        "messaging_product": "whatsapp",
        "to": to_number,
        "type": "text",
        "text": {"body": message[:1000]} # Límite de caracteres por seguridad
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code in [200, 201]:
            return "WhatsApp enviado (Meta)."
        return f"Error Meta: {response.text}"
    except Exception as e:
        return f"Excepción: {e}"

In [21]:
@tool("Leer correo")
def read_emails() -> str:
    """
    Lee correos de hoy filtrando 'category:primary' para evitar spam/promociones.
    Returns: JSON con lista de correos.
    """
    try:
        creds = authenticate_google_services()
        service = build("gmail", "v1", credentials=creds)

        hoy = datetime.now().strftime("%Y/%m/%d")
        mañana = (datetime.now() + timedelta(days=1)).strftime("%Y/%m/%d")

        # MEJORA: Filtramos por category:primary para reducir ruido
        query = f"after:{hoy} before:{mañana}"

        results = service.users().messages().list(userId='me', q=query, maxResults=50).execute()
        messages = results.get('messages', [])

        correos = []
        if not messages:
            return json.dumps({"correos": [], "mensaje": "Bandeja de entrada absolutamente vacía hoy."}, ensure_ascii=False)
        for msg in messages:
            txt = service.users().messages().get(userId='me', id=msg['id']).execute()
            headers = txt.get('payload', {}).get('headers', [])
            snippet = txt.get('snippet', '')

            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(Sin Asunto)')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '(Desconocido)')

            # Limpiamos el remitente para que sea más legible (ej: "Google <no-reply...>" -> "Google")
            sender_name = sender.split('<')[0].strip().replace('"', '')

            correos.append({"remitente": sender, "asunto": subject, "resumen": txt.get('snippet', '')[:200]})

        if not correos: return json.dumps({"correos": [], "mensaje": "Sin correos importantes hoy."}, ensure_ascii=False)
        return json.dumps({"correos": correos}, indent=2, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})

In [22]:
@tool("Calendario")
def get_todays_agenda() -> str:
    """Obtiene eventos de hoy del calendario principal."""
    try:
        creds = authenticate_google_services()
        service = build("calendar", "v3", credentials=creds)

        now = datetime.now()
        time_min = now.replace(hour=0, minute=0, second=0).isoformat() + 'Z'
        time_max = now.replace(hour=23, minute=59, second=59).isoformat() + 'Z'

        events_result = service.events().list(
            calendarId='primary', timeMin=time_min, timeMax=time_max,
            singleEvents=True, orderBy='startTime'
        ).execute()

        agenda = []
        for event in events_result.get('items', []):
            start = event['start'].get('dateTime', event['start'].get('date'))
            agenda.append({
                "titulo": event.get('summary', 'Sin título'),
                "inicio": start,
                "ubicacion": event.get('location', 'N/A')
            })

        if not agenda: return json.dumps({"agenda": [], "mensaje": "Agenda libre."}, ensure_ascii=False)
        return json.dumps({"agenda": agenda}, indent=2, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})


In [23]:
@tool("Tareas")
def get_todays_tasks() -> str:
    """Obtiene tareas que vencen hoy."""
    try:
        creds = authenticate_google_services()
        service = build("tasks", "v1", credentials=creds)

        hoy_str = datetime.now().strftime("%Y-%m-%d") # Comparación por string es más segura en ISO
        tareas = []

        # Listamos todas las listas de tareas
        tasklists = service.tasklists().list(maxResults=5).execute()

        for lista in tasklists.get('items', []):
            tasks = service.tasks().list(tasklist=lista['id'], showCompleted=False).execute()
            for t in tasks.get('items', []):
                due = t.get('due')
                if due and due.startswith(hoy_str): # Chequeo simple de fecha
                    tareas.append({
                        "lista": lista['title'],
                        "titulo": t['title'],
                        "notas": t.get('notes', '')
                    })

        if not tareas: return json.dumps({"tareas": [], "mensaje": "Sin tareas para hoy."}, ensure_ascii=False)
        return json.dumps({"tareas": tareas}, indent=2, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})

In [24]:
@tool("Transporte")
def inc_transport():
  """Obtiene problemas en el transporte."""
  URL = 'https://tmpmurcia.es/ultima.asp'

  try:
    parsed = urlparse(URL)
    dominio = f"{parsed.scheme}://{parsed.netloc}/"  # esquema + dominio

    resp = requests.get(URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    for a in soup.find_all('a', href=True): # Iterate through all <a> tags
      if "Cuerpo.asp?codigo=" in a['href']:
        enlace = dominio + a['href']
        break

    r = requests.get(enlace)
    r.raise_for_status()
    sub_soup = BeautifulSoup(r.text, 'html.parser')

    # Buscar imagen en /fotos/noticias/
    img_tag = sub_soup.find('img', src=lambda x: x and '/fotos/noticias/' in x)
    if not img_tag:
        return {
            'pagina': enlace,
            'error': 'No se encontró imagen en /fotos/noticias/'
            }

    img_url = img_tag['src']
    if not img_url.startswith('http'):
        img_url = f"https://tmpmurcia.es/{img_url.lstrip('/')}"

    # Descargar imagen
    img_resp = requests.get(img_url)
    img_resp.raise_for_status()
    imagen = Image.open(BytesIO(img_resp.content))

    # OCR
    texto = pytesseract.image_to_string(imagen, lang='spa')
    lineas = texto.splitlines()

    # Buscar línea 44
    ocurrencias_44 = [line for line in lineas if '44' in line]

    return {
        'pagina': enlace,
        'imagen': img_url,
        'ocurrencias_44': ocurrencias_44,
        'texto': texto,
    }

  except Exception as e:
    return {
        'pagina': enlace,
        'error': str(e)
        }

# 4. CONFIGURACIÓN DE AGENTES Y TAREAS (CREWAI)

# 🕵️‍♂️ El Equipo (The Crew): Roles y Capacidades

El sistema está compuesto por 6 agentes especializados. A continuación se detalla la función técnica de cada uno:

### 1. 🚍 Transport & Incident Agent (El "MVP" Técnico)
**Rol:** Analista de incidencias de transporte.
**Tecnología Avanzada:** Este agente no solo lee texto; **"ve"** la información.
*   **Capacidad de Visión (OCR):** Utiliza una herramienta personalizada que navega a la web de transporte de Murcia, localiza la imagen del boletín diario (que cambia de URL dinámicamente), la descarga y aplica **Tesseract OCR (Optical Character Recognition)** para extraer el texto incrustado en la imagen.
*   **Lógica:** Analiza el texto extraído buscando menciones específicas a líneas de autobús críticas (ej. línea 44) para alertar sobre retrasos.

### 2. 📧 & 📅 Communication & Schedule Agents
**Rol:** Gestión de agenda y correo.
**Ingeniería de Prompts (Anti-Hallucination):** 
*   Se ha implementado un *Prompting* defensivo estricto ("Guardrails").
*   Al `calendar_agent` se le prohíbe explícitamente "inventar" reuniones de relleno (como "Almuerzo" o "Planificación") si la API devuelve una lista vacía. Esto garantiza que el reporte final sea 100% factual y confiable para la toma de decisiones.

### 3. 📈 Market Intelligence Agent
**Rol:** Analista financiero.
**Herramientas:** 
*   `yfinance`: Para extracción de datos numéricos en tiempo real (Tickers: REP.MC).
*   `feedparser` + `requests`: Para realizar scraping de noticias RSS de Google News, filtrando por relevancia semántica.

### 4. 📝 Briefing & Editor Agent
**Rol:** Jefe de Gabinete (Chief of Staff).
**Función:** No genera nueva información, sino que sintetiza los outputs de los agentes anteriores.
*   Decide qué es relevante (ej. si no hay correos urgentes, omite la sección).
*   Utiliza herramientas de **API Gateway** para enviar el mensaje final formateado a Telegram y WhatsApp.

In [25]:
# Agentes con roles más definidos y anti-alucinación
mail_agent = Agent(
    role="Analista de Comunicaciones General",
    goal="Revisar TODO el correo entrante y reportar ofertas interesantes, alertas de seguridad y notificaciones.",
    backstory="Eres un asistente EA de alto nivel. Tu trabajo es que el usuario no se pierda ninguna oportunidad, ni alerta importante.",
    llm=get_llm(),
    tools=[read_emails],
    verbose=False
)

In [26]:
calendar_agent = Agent(
    role="Lector de Datos de Calendario",
    goal="Extraer y listar LITERALMENTE los eventos que devuelve la herramienta. NO modificar ni inventar nada.",
    backstory="""Eres un programa automatizado, no un humano.
    Tu única función es leer la salida JSON de la herramienta de calendario y listarla tal cual.
    Tienes PROHIBIDO inventar reuniones como 'Planificación', 'Almuerzos' o 'Entrevistas'.
    Si la herramienta dice 'Turno Tardes', tú escribes 'Turno Tardes'.
    Si la herramienta no devuelve nada, tú dices 'No hay eventos'.""",
    llm=get_llm(),
    tools=[get_todays_agenda],
    verbose=False
)

In [27]:
task_agent = Agent(
    role="Lector de tareas",
    goal="Listar tareas vencidas, para hoy o los próximos dos días. No sugerir tareas genéricas.",
    backstory="Solo te importan las tareas registradas en el sistema (Google Tasks).",
    llm=get_llm(),
    tools=[get_todays_tasks],
    verbose=False
)

In [28]:
analyst_agent = Agent(
    role="Analista de Inteligencia de Mercado",
    goal="Proveer datos duros sobre 'El Corte Inglés' y 'Repsol'.",
    backstory="Analista bursátil. Te basas en datos, no en especulaciones. Usas las herramientas para buscar precios y noticias frescas.",
    llm=get_llm(),
    tools=[get_stock_price, get_financial_news],
    verbose=False
)

In [29]:

transport_agent = Agent(
    role="Analista de incidencias de transporte",
    goal="Identificar y listar los problemas de transporte reportados para la fecha actual.",
    backstory="Este agente se centra exclusivamente en los problemas que el sistema indica, y únicamente si corresponden al día en curso.",
    llm=get_llm(),
    tools=[inc_transport],
    verbose=False
)

In [30]:
briefing_agent = Agent(
    role="Redactor de Reporte Factual",
    goal="Consolidar toda la información veraz en un reporte ejecutivo.",
    backstory="""
    Eres el Jefe de Gabinete. Tu responsabilidad es generar el 'Briefing' diario.
    REGLA DE ORO: NO INVENTES INFORMACIÓN.
    Si no hay correos, dí 'Sin correos.'.
    Si no hay tareas, NO inventes 'Aprobar pago'.
    Usa estrictamente los datos que te pasan los otros agentes.
    Jamás agregues eventos que no te hayan pasado explícitamente los otros agentes.
    """,
    llm=get_llm(),
    tools=[send_pushover, send_telegram],
    verbose=False
)

# Definición de Tareas (Tasks)

In [31]:
fecha_hoy = datetime.now().strftime('%d/%m/%Y')

In [32]:
mail_scan = Task(
    description=f"Lee TODOS los correos de hoy ({fecha_hoy}). Identifica cualquier notificación relevante y categorizala.",
    expected_output="Lista JSON o bullet points de correos categorizados.",
    agent=mail_agent
)

In [33]:
calendar_scan = Task(
    description=f"""Obtén la agenda de hoy ({fecha_hoy}). MIRA los datos reales que devuelve. GENERA una lista usando SOLO esos datos.
    ⚠️ ALERTA DE SEGURIDAD:
    - NO INVENTES "Reuniones de Estrategia".
    - NO INVENTES "Almuerzos de equipo".
    - Si el JSON recuperado contiene eventos reales, ÚSALOS.
    - Si ignoras los datos reales ("Turno Tardes") para inventar datos falsos, fallarás tu misión.""",
    expected_output="Resumen cronológico de eventos reales del calendario.",
    agent=calendar_agent
)

In [34]:
task_scan = Task(
    description=f"Lista las tareas con vencimiento hoy ({fecha_hoy}).",
    expected_output="Lista de tareas pendientes reales extraídas de Google Tasks.",
    agent=task_agent
)

In [35]:
analyst_scan = Task(
    description="1. Obtén precio de REP.MC. 2. Busca noticias de 'El Corte Inglés'.",
    expected_output="Datos financieros y resumen de 3 noticias principales.",
    agent=analyst_agent
)

In [36]:
transport_scan = Task(
    description="Verificar si existen incidencias en el transporte para la fecha actual.",
    expected_output="Generar una alerta crítica únicamente si se detectan incidencias correspondientes al día en curso.",
    agent=transport_agent
)

In [37]:
task_notify = Task(
    description=f"""
    Genera el BRIEFING -{fecha_hoy}.

    ESTRUCTURA OBLIGATORIA:
    1. 📅 Agenda: Resumen de eventos (Solo si existen).
    2. ✅ Tareas: Lista de tareas (Solo si existen).
    3. 📧 Correos Clave: (Solo si hay importantes).
    4. 📈 Mercado: Precio REP.MC y 1 titular clave de El Corte Inglés.
    5. 🚚 Transporte: Incidencias de transporte.

    NOTA: NO inventes datos. NO inventes tareas. NO inventes eventos. NO inventes correos. NO inventes noticias. Las incidencias de trasporte son CRITICAS.

    ADVERTENCIA: NO agregues secciones de 'Prioridades Críticas' si las tareas están vacías. Sé fiel a los datos.
    Al final, envía el resumen por Pushover y WhatsApp.
    """,
    expected_output="Reporte enviado y texto final de confirmación.",
    agent=briefing_agent,
    context=[mail_scan, calendar_scan, task_scan, analyst_scan, transport_scan]
)

# Equipo

In [38]:
morning_crew = Crew(
    agents=[mail_agent, calendar_agent, task_agent, analyst_agent, transport_agent, briefing_agent],
    tasks=[mail_scan, calendar_scan, task_scan, analyst_scan, transport_scan, task_notify],
    verbose=False
)

# 5. EJECUCIÓN PRINCIPAL

## 🔄 Flujo de Ejecución (Pipeline)

1.  **Ingesta:** Los agentes de Correo, Calendario y Tareas consultan las APIs de Google.
2.  **Investigación:** El agente de Mercado y Transporte ejecutan scripts de Python externos (Scraping/OCR).
3.  **Procesamiento:** Cada agente devuelve un JSON estructurado o un resumen en lenguaje natural.
4.  **Síntesis:** El `Briefing Agent` toma el contexto de todas las tareas anteriores.
5.  **Entrega:** Se dispara la notificación al smartphone del usuario.

In [39]:
def run_shopping_assistant():
    logger.info("🚀 Iniciando Asistente Ejecutivo IA...")
    try:
        result = morning_crew.kickoff()
        logger.info("✅ Ejecución finalizada con éxito.")
        return result
    except Exception as e:
        logger.critical(f"🔥 Error crítico en la ejecución del Crew: {e}")
        return str(e)

In [40]:
if __name__ == "__main__":
    resultado_crew = run_shopping_assistant()

2025-11-30 07:43:54,148 - [INFO] - 🚀 Iniciando Asistente Ejecutivo IA...
2025-11-30 07:43:54,177 - [INFO] - AFC is enabled with max remote calls: 10.
2025-11-30 07:43:56,543 - [INFO] - 🔄 Token refrescado automáticamente
2025-11-30 07:43:56,553 - [INFO] - file_cache is only supported with oauth2client<4.0.0
2025-11-30 07:44:05,118 - [INFO] - AFC is enabled with max remote calls: 10.
2025-11-30 07:44:16,544 - [INFO] - AFC is enabled with max remote calls: 10.
2025-11-30 07:44:18,675 - [INFO] - file_cache is only supported with oauth2client<4.0.0
2025-11-30 07:44:20,399 - [INFO] - AFC is enabled with max remote calls: 10.
2025-11-30 07:44:21,298 - [INFO] - AFC is enabled with max remote calls: 10.
2025-11-30 07:44:24,332 - [WARNING] - Warning: there are non-text parts in the response: ['thought_signature'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-11-30 07:44:24,333 - [WARNING] - Warning: ther

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

# 6. VISUALIZACIÓN DE DATOS (POST-EJECUCIÓN)

In [41]:
# COmprobación
print("\n" + "="*40)
print("📱 VISTA PREVIA DEL MENSAJE ENVIADO")
print("="*40 + "\n")

# Verificamos si la ejecución anterior guardó el resultado
if 'resultado_crew' in locals() and resultado_crew:
    # Convertimos el resultado a string (CrewAI devuelve un objeto CrewOutput)
    reporte_texto = str(resultado_crew)

    # 1. Mostrar renderizado (Bonito)
    display(Markdown(reporte_texto))

    # 2. (Opcional) Mostrar caracteres raw por si hay errores de formato
    # print("\n--- Texto Plano ---\n", reporte_texto)
else:
    print("⚠️ No se encontró el reporte ('resultado_crew').")
    print("Asegúrate de haber ejecutado la celda principal: resultado_crew = run_shopping_assistant()")


📱 VISTA PREVIA DEL MENSAJE ENVIADO



BRIEFING DIARIO - 30/11/2025

1. 📅 Agenda
Sin eventos programados para hoy.

2. ✅ Tareas
- Pagar factura de luz
- Revisar propuesta de proyecto X

3. 📧 Correos Clave
- Oportunidad de Carrera: Infoempleo notifica más de 2 ofertas de empleo de Desarrollador/a de Software Senior especializado en sistemas SCADA.
- Empleo Público: Novedades del Ministerio de Justicia sobre Auxilio Judicial (acceso libre, estabilización).
- Informativo AI: Marktechpost reporta lanzamientos clave: NVIDIA AI Releases Orchestrator-8B y DeepSeek AI Releases DeepSeekMath-V2.

4. 📈 Mercado
- REP.MC (Repsol): Precio actual 15.98 EUR. Variación diaria +1.46%.
- El Corte Inglés: Marta Álvarez deja la presidencia de El Corte Inglés tras renovar su mandato en julio y la sustituye su hermana.

5. 🚚 Transporte
Sin incidencias de transporte detectadas.

Reporte enviado y texto final de confirmación.

In [42]:
    # Esto realiza llamadas extra a las APIs para generar las tablas visuales
    # Útil para depuración o reportes visuales en Notebooks.

    print("\n" + "="*50 + "\n GENERANDO REPORTE VISUAL (TABLAS) \n" + "="*50)

    try:
        # Funciones helper para extracción segura
        def safe_json_load(tool_func, **kwargs):
            try:
                res = tool_func._run(**kwargs)
                return json.loads(res)
            except:
                return {}

        raw_emails = safe_json_load(read_emails).get("correos", [])
        raw_agenda = safe_json_load(get_todays_agenda).get("agenda", [])
        raw_tasks  = safe_json_load(get_todays_tasks).get("tareas", [])
        raw_news   = safe_json_load(get_financial_news, busqueda="El Corte Inglés").get("news", [])
        raw_stock  = safe_json_load(get_stock_price, symbol="REP.MC").get("stock", {})

        # Mostrar Markdown
        reporte_md = f"""
        # 📊 Tablero de Control - {datetime.now().strftime('%Y-%m-%d')}
        """
        display(Markdown(reporte_md))

        if raw_agenda:
            display(Markdown("### 📅 Agenda"))
            display(pd.DataFrame(raw_agenda))
        else:
            display(Markdown("📅 **Agenda:** Sin eventos."))

        if raw_tasks:
            display(Markdown("### ✅ Tareas"))
            display(pd.DataFrame(raw_tasks))

        if raw_emails and "mensaje" not in raw_emails[0]:
            display(Markdown("### 📧 Correos Recientes"))
            display(pd.DataFrame(raw_emails))

        if raw_stock:
            display(Markdown(f"### 📈 Mercado ({raw_stock.get('symbol', 'N/A')})"))
            df_stock = pd.DataFrame([raw_stock])
            display(df_stock[['currentPrice', 'dayHigh', 'dayLow', 'regularMarketChangePercent']])

        if raw_news:
            display(Markdown("### 📰 Noticias ECI"))
            display(pd.DataFrame(raw_news)[['titulo', 'fuente', 'fecha']])

    except Exception as e:
        logger.error(f"Error generando visualización final: {e}")

2025-11-30 07:45:18,502 - [INFO] - file_cache is only supported with oauth2client<4.0.0



 GENERANDO REPORTE VISUAL (TABLAS) 


2025-11-30 07:45:26,548 - [INFO] - file_cache is only supported with oauth2client<4.0.0
2025-11-30 07:45:27,904 - [INFO] - file_cache is only supported with oauth2client<4.0.0



    # 📊 Tablero de Control - 2025-11-30
    

📅 **Agenda:** Sin eventos.

...

,remitente,asunto,resumen
0,Primeriti <noreply@mc.primeriti.es>,¡Último día!🖤Disfruta del -20% adicional,"Punto Blanco, Niza, Izas y más Si no puedes vi..."
1,PcComponentes <noreply@pccomponentes.com>,Este Black Friday se acaba YA: no lo dejes esc...,Los descuentos están a punto de terminar y que...
2,Chollometro <no-reply@chollometro.com>,Tu alerta 'proteínas' alert - 'Proteína de sue...,logotipo del Chollometro logotipo del Chollome...
3,PcComponentes <noreply@pccomponentes.com>,Este Black Friday se acaba YA: no lo dejes esc...,Los descuentos están a punto de terminar y que...
4,UNIQLO <emails@euenews.uniqlo.com>,Black Friday | Plumas sin costuras en oferta,"Y más descuentos de Black Friday, en tiendas y..."
5,Tom Parfum <info@tomparfum.com>,La elección de cada mañana,"Tu perfume, tu decisión ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ..."
6,Infoempleo <notificador@notificador.infoempleo...,Más de 2 ofertas de empleo de Desarrollador/a ...,30 de noviembre de 2025 Infoempleo Digital Mur...
7,Google Calendar <calendar-notification@google....,No tienes ningún evento programado para hoy.,"Google Calendar José Martínez Ruiz, no tienes ..."
8,Marktechpost AI <marktechpost-newsletter@mail....,NVIDIA AI Releases Orchestrator-8B | DeepSeek ...,"November 29, 2025 | Read Online Here is your t..."
9,The Dua Brand <support@theduabrand.com>,"🎮💥 READY, SET, CLICK! Black Friday Game + NEW ...",Up to 41% Off on Gift Sets &amp; match your Bl...


### 📈 Mercado (REP.MC)

,currentPrice,dayHigh,dayLow,regularMarketChangePercent
0,15.98,15.99,15.8,1.460314


### 📰 Noticias ECI

,titulo,fuente,fecha
0,“La empresa no nos respeta”: personas trabajad...,El Salto,"Fri, 28 Nov 2025 13:10:00 GMT"
1,Ni Mercadona ni El Corte Inglés: el supermerca...,losreplicantes.com,"Sat, 29 Nov 2025 08:30:45 GMT"
2,Marta Álvarez deja la presidencia de El Corte ...,EL PAÍS,"Wed, 26 Nov 2025 16:21:53 GMT"
3,De vendedora en El Corte Inglés a presidenta d...,El Periódico,"Thu, 27 Nov 2025 12:36:12 GMT"
4,"Sorpresa en El Corte Inglés: ""relevo ordenado""...",El Confidencial,"Thu, 27 Nov 2025 04:00:00 GMT"
